<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/No_supervisados_Ejercicio_1__pca_mall_customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# No supervisados - Ejercicio 1: pca_mall_customers.ipynb

Este notebook es **I do**. Está resuelto: ejecuta las celdas para ver paso a paso cómo aplicar PCA sobre el dataset *Mall Customers*, cómo elegir el número de componentes, cómo medir la pérdida por reconstrucción y cómo usar PCA como preprocesado antes de clustering. Lee las explicaciones en cada celda y observa los resultados.

## Objetivos

- Cargar el dataset *Mall Customers* (con fallback sintético si no está disponible).
- Estandarizar las variables y calcular PCA completo.
- Analizar varianza explicada y seleccionar componentes que retengan el 90% de la varianza.
- Medir el error de reconstrucción (MSE) para distintos números de componentes.
- Comparar el efecto de aplicar PCA antes de KMeans (inertia) y visualizar en 2D.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

np.random.seed(42)


### 1) Carga de datos

Intento cargar desde un par de URLs públicas. Si no se puede, genero un dataset sintético con la misma estructura para que puedas ejecutar el notebook sin dependencias de red.

In [ ]:
url = 'https://raw.githubusercontent.com/dtoralg/INESDI_Data-Science_ML_IA/refs/heads/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/Mall_Customers.csv'

df = pd.read_csv(url)
print('Cargado desde', url)


### 2) Exploración rápida

Comprueba nulos, tipos y estadísticas resumidas; esto te ayudará a decidir qué columnas usar para PCA.

In [ ]:
print('Porcentaje de nulos por columna:')
print((df.isnull().mean()*100).round(2))
print('\nTipos de columnas:')
print(df.dtypes)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print('\nColumnas numéricas detectadas:', num_cols)

display(df[num_cols].describe().T)

### 3) Preparación: encoding y selección de features

Codifico `Gender` y selecciono las variables numéricas relevantes para PCA: `Age`, `Annual Income (k$)` y `Spending Score (1-100)`. Puedes añadir más columnas si lo deseas, pero al menos estas tres son representativas para segmentación.

In [ ]:
df_proc = df.copy()
if 'Gender' in df_proc.columns:
    le = LabelEncoder()
    df_proc['Gender_enc'] = le.fit_transform(df_proc['Gender'].astype(str))

features = [c for c in ['Age','Annual Income (k$)','Spending Score (1-100)'] if c in df_proc.columns]
print('Features seleccionadas:', features)
X = df_proc[features].copy()
display(X.head())

### 4) Escalado

Estandariza las features para que cada variable contribuya por igual a las componentes principales.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=features)
display(X_scaled_df.describe().T)

### 5) Cálculo de PCA y varianza explicada

Ajusto PCA completo (sin limitar el nº de componentes) y muestro la varianza explicada por cada componente y la acumulada. Esto te permite decidir cuántas componentes retener.

In [ ]:
pca_full = PCA()
X_pca_full = pca_full.fit_transform(X_scaled)
explained = pca_full.explained_variance_ratio_
cumulative = np.cumsum(explained)

explained_df = pd.DataFrame({'pc': np.arange(1, len(explained)+1), 'explained_variance': explained, 'cumulative': cumulative})
explained_df.index = explained_df['pc']
display(explained_df[['explained_variance','cumulative']])

plt.figure(figsize=(8,4))
plt.plot(explained_df['pc'], explained_df['explained_variance'], marker='o', label='Individual')
plt.plot(explained_df['pc'], explained_df['cumulative'], marker='o', label='Cumulative')
plt.xlabel('Componente principal')
plt.ylabel('Varianza explicada')
plt.xticks(explained_df['pc'])
plt.legend()
plt.title('Scree plot y varianza acumulada')
plt.grid(True)
plt.show()

### 6) Elegir número de componentes para retener el 90% de la varianza

Calculo cuántas componentes necesito para retener el 90% de la varianza y creo la transformación correspondiente.

In [ ]:
target_var = 0.90
n_components_90 = int(np.argmax(cumulative >= target_var) + 1)
print(f'Componentes necesarios para retener {int(target_var*100)}% de varianza: {n_components_90}')

pca90 = PCA(n_components=n_components_90)
X_pca_90 = pca90.fit_transform(X_scaled)
print('Shape after PCA (90%):', X_pca_90.shape)

### 7) Reconstrucción y error de reconstrucción

Reconstruyo los datos desde diferentes números de componentes y mido el MSE para entender la pérdida de información que introduce la reducción dimensional.

In [ ]:
mse_records = []
for k in range(1, X_scaled.shape[1]+1):
    pca_k = PCA(n_components=k)
    Xk = pca_k.fit_transform(X_scaled)
    Xk_rec = pca_k.inverse_transform(Xk)
    mse = mean_squared_error(X_scaled, Xk_rec)
    mse_records.append({'k': k, 'mse': mse})

mse_df = pd.DataFrame(mse_records).set_index('k')
display(mse_df)

plt.figure(figsize=(6,4))
plt.plot(mse_df.index, mse_df['mse'], marker='o')
plt.xlabel('n_components')
plt.ylabel('MSE reconstrucción')
plt.title('Error de reconstrucción vs nº componentes')
plt.grid(True)
plt.show()

### 8) Visualización en 2D (PCA) y observaciones

Proyecto los datos en las dos primeras componentes principales y uso un gradiente con `Spending Score` para detectar patrones. Además muestro cómo se verían centroides de un KMeans de ejemplo en este espacio para ayudarte a interpretar los clusters.

In [ ]:
pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=df_proc['Spending Score (1-100)'] if 'Spending Score (1-100)' in df_proc.columns else X_scaled[:, -1], cmap='viridis', s=60)
plt.colorbar(label='Spending Score (1-100)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Datos proyectados en las 2 primeras componentes (color = Spending Score)')
plt.show()

# Centroides de ejemplo
km_example = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_km_ex = km_example.fit_predict(X_scaled)
centroids = km_example.cluster_centers_
centroids_pca = pca2.transform(centroids)

plt.figure(figsize=(8,6))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=labels_km_ex, cmap='tab10', s=40, alpha=0.6)
plt.scatter(centroids_pca[:,0], centroids_pca[:,1], c='black', s=150, marker='X', label='centroides')
plt.title('KMeans (k=4) visualizado en PCA 2D con centroides')
plt.legend()
plt.show()

### 9) PCA como preprocesado antes de clustering

Comparo la inercia de KMeans ejecutado sobre los datos escalados frente a la representación reducida por PCA (2 componentes y n_components_90). Esto te da una idea de cómo afecta PCA a la estructura de clusters.

In [ ]:
km_orig = KMeans(n_clusters=4, random_state=42, n_init=10).fit(X_scaled)
km_pca2 = KMeans(n_clusters=4, random_state=42, n_init=10).fit(pca2.transform(X_scaled))
km_pca90 = KMeans(n_clusters=4, random_state=42, n_init=10).fit(X_pca_90)

print('Inertia original (scaled):', km_orig.inertia_)
print('Inertia PCA 2D:', km_pca2.inertia_)
print('Inertia PCA 90%:', km_pca90.inertia_)

### 10) Conclusión

Has calculado PCA, medido varianza explicada y evaluado la pérdida de información por reconstrucción. Para visualización se usan 2 componentes; para preprocesado antes de clustering usé las componentes necesarias para retener el 90% de varianza. PCA reduce dimensionalidad y ruido y acelera el clustering, aunque introduce una pérdida medida por el MSE de reconstrucción.